# Importing Dependencies

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

# Loading The Model

In [2]:
interpreter = tf.lite.Interpreter(model_path='singlepose-lightning.tflite')
interpreter.allocate_tensors()

# Drawing Keypoints

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [4]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Drawing Edges

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

# Making Detections

In [6]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[[[[0.73733795 0.5981169  0.27064577]
   [0.7122793  0.6219722  0.26748595]
   [0.71041965 0.58604836 0.26062012]
   [0.7148423  0.6827437  0.2144423 ]
   [0.70552766 0.6008994  0.21069361]
   [0.8198508  0.73428243 0.13788398]
   [0.792695   0.64230037 0.2706469 ]
   [0.8516606  0.70533884 0.05501606]
   [0.82006    0.651275   0.08437263]
   [0.825889   0.65958846 0.07888775]
   [0.7975352  0.5918062  0.09673894]
   [0.5646703  0.97801316 0.12807062]
   [0.9007496  0.6928045  0.03953436]
   [1.001127   0.6526301  0.00977663]
   [0.85547173 0.55165714 0.03182282]
   [0.99774474 0.58760405 0.01238717]
   [1.0008217  0.5212098  0.00976356]]]]
[[[[0.7305104  0.5850171  0.28887138]
   [0.7112758  0.6175079  0.20145786]
   [0.7048441  0.5784579  0.3297709 ]
   [0.7069719  0.69606847 0.19476864]
   [0.6939082  0.5840531  0.16011131]
   [0.80178213 0.73676074 0.13229024]
   [0.7589948  0.5958392  0.23569945]
   [0.8441645  0.7231225  0.08080905]
   [0.7991897  0.565318   0.13821211]
   [0.823

[[[[0.81933975 0.8426497  0.3120992 ]
   [0.77760756 0.84909457 0.26058966]
   [0.77360344 0.77665204 0.35091126]
   [0.71536416 0.78486824 0.154147  ]
   [0.719982   0.6529782  0.22102946]
   [0.77361846 0.78435874 0.16061331]
   [0.8055866  0.6188893  0.14272967]
   [0.8528689  0.8618881  0.05686569]
   [0.8511187  0.5254618  0.07091653]
   [0.8601819  0.88621604 0.04871438]
   [0.86080396 0.71198976 0.02992788]
   [0.78084934 0.72865075 0.01923086]
   [0.8252248  0.6402129  0.05766441]
   [0.84995604 0.9107305  0.01913741]
   [0.73832786 0.1768788  0.08591232]
   [0.84470475 0.91283596 0.01697719]
   [0.8481206  0.7004124  0.01450108]]]]
[[[[0.8123847  0.8415159  0.28612298]
   [0.77262616 0.8517879  0.21221246]
   [0.7693316  0.80151314 0.36053944]
   [0.72605747 0.8328774  0.16700768]
   [0.71905184 0.65190816 0.17751166]
   [0.8067566  0.8298709  0.19411786]
   [0.80693364 0.5988799  0.16383335]
   [0.8507551  0.8846065  0.09590708]
   [0.83569163 0.53952235 0.1071223 ]
   [0.845

[[[[0.8008065  0.80558074 0.2714829 ]
   [0.77008414 0.81674284 0.18317112]
   [0.76421416 0.7568538  0.29752672]
   [0.7326058  0.7857313  0.19479491]
   [0.7277865  0.64667416 0.3173618 ]
   [0.8169908  0.7895318  0.1484836 ]
   [0.81749785 0.58210105 0.20040664]
   [0.8533883  0.8399956  0.07420105]
   [0.8338739  0.53438294 0.092961  ]
   [0.84884405 0.83403486 0.09452905]
   [0.8527932  0.63451684 0.04001138]
   [0.8385955  0.7394372  0.05533614]
   [0.8499894  0.6225745  0.05530199]
   [0.8386693  0.82575536 0.02792316]
   [0.82827485 0.5302663  0.03137658]
   [0.8437149  0.8276429  0.04092101]
   [0.83186495 0.5983086  0.03629924]]]]
[[[[0.8191936  0.8282525  0.24527228]
   [0.7785107  0.8360739  0.20527941]
   [0.77426994 0.78157115 0.2579135 ]
   [0.7242591  0.78299916 0.23223709]
   [0.72090834 0.6694169  0.33993888]
   [0.79593325 0.78608847 0.18748192]
   [0.8062632  0.6039803  0.19259985]
   [0.85208297 0.87200564 0.07860018]
   [0.8369142  0.52767694 0.07022841]
   [0.849

[[[[0.7972263  0.79111546 0.22221757]
   [0.76675266 0.8093764  0.14733209]
   [0.7563983  0.7342438  0.28017217]
   [0.7256967  0.7934704  0.18830968]
   [0.7068906  0.62046087 0.16165319]
   [0.78544354 0.8139424  0.226109  ]
   [0.7810532  0.5641841  0.29245734]
   [0.85349154 0.87839115 0.11019   ]
   [0.8570542  0.5171585  0.16440207]
   [0.85063094 0.8568129  0.11659033]
   [0.85480607 0.61573476 0.04144329]
   [0.8342917  0.75354666 0.09715804]
   [0.8521301  0.6023514  0.05211371]
   [0.83615965 0.8604289  0.02998063]
   [0.8236989  0.50088966 0.02683024]
   [0.84272474 0.82252705 0.05104858]
   [0.8304965  0.60480726 0.02234212]]]]
[[[[0.78663486 0.83714885 0.34719324]
   [0.7496768  0.8350458  0.29435167]
   [0.7516197  0.7986121  0.3531223 ]
   [0.71547145 0.8002181  0.1800462 ]
   [0.71852076 0.6801111  0.30086058]
   [0.789453   0.8191478  0.24300043]
   [0.7913989  0.5805293  0.22444719]
   [0.8497579  0.86427134 0.09354287]
   [0.847257   0.5203188  0.14995976]
   [0.851

[[[[0.7859925  0.7885647  0.20349513]
   [0.7587205  0.8064439  0.15128021]
   [0.7568716  0.752753   0.35284746]
   [0.7218902  0.78197145 0.21075976]
   [0.7081219  0.62569577 0.19611274]
   [0.78849554 0.8075065  0.21430252]
   [0.78009313 0.56437665 0.28281122]
   [0.8530518  0.8824715  0.08862514]
   [0.84492695 0.53253967 0.12413895]
   [0.8541548  0.84399784 0.09847078]
   [0.84968823 0.608392   0.05542612]
   [0.85705394 0.7351979  0.07951584]
   [0.8648919  0.603783   0.07299105]
   [0.8401625  0.83943725 0.0230425 ]
   [0.8281111  0.5221035  0.03052294]
   [0.84753364 0.8155347  0.04446869]
   [0.8301946  0.62724483 0.03074078]]]]
[[[[0.7703695  0.7739622  0.16538633]
   [0.7401723  0.78897494 0.12998459]
   [0.7220818  0.65975034 0.36305827]
   [0.719367   0.7516123  0.17182307]
   [0.69908446 0.6063147  0.16742824]
   [0.8084359  0.79581684 0.18055606]
   [0.79498136 0.5487118  0.25612354]
   [0.8528239  0.8508715  0.08323943]
   [0.8562937  0.5181507  0.12107221]
   [0.849

...

[[[[0.40809163 0.6161257  0.785259  ]
   [0.32359648 0.67669594 0.78312135]
   [0.3188614  0.5381663  0.75264174]
   [0.3524359  0.70445    0.43954706]
   [0.34557498 0.42127067 0.7124431 ]
   [0.65312356 0.8090775  0.55219805]
   [0.62095153 0.23095691 0.46294853]
   [0.87182236 0.8715669  0.01641497]
   [0.8632509  0.11338504 0.0976479 ]
   [0.8584511  0.88054293 0.00885125]
   [0.72229624 0.33359405 0.01274414]
   [0.44507903 0.53851855 0.01303423]
   [0.88607687 0.05232069 0.02589383]
   [0.67227143 0.9550903  0.00718829]
   [0.66426176 0.16964416 0.0417444 ]
   [0.5639148  1.0228562  0.00583391]
   [0.75996274 0.02391378 0.02489621]]]]
[[[[0.42032787 0.6231043  0.5364947 ]
   [0.33602497 0.6803229  0.7196294 ]
   [0.32579556 0.54339206 0.621075  ]
   [0.3631539  0.70303    0.56227887]
   [0.35467935 0.42063087 0.6674957 ]
   [0.6650787  0.80982935 0.667642  ]
   [0.6541916  0.21215862 0.43167353]
   [0.8945062  0.88764733 0.01717091]
   [0.881439   0.12541723 0.04537084]
   [0.858

[[[[0.45157298 0.61624384 0.8193333 ]
   [0.3625545  0.6845466  0.77125806]
   [0.3559452  0.5361812  0.7321133 ]
   [0.38289762 0.7219161  0.6394944 ]
   [0.38411054 0.4234091  0.8121571 ]
   [0.6778356  0.8118543  0.80587727]
   [0.61160386 0.24284153 0.8240439 ]
   [0.91873574 0.9026012  0.01428978]
   [0.860022   0.10560121 0.24218592]
   [0.88217545 0.984116   0.01766481]
   [0.8665246  0.11642681 0.10813857]
   [0.4856885  0.56209433 0.01346881]
   [0.98833174 0.28876016 0.01620102]
   [0.74688995 0.95977634 0.00593885]
   [0.60101223 0.24893665 0.01817954]
   [0.65358037 1.0241112  0.00427226]
   [0.86953896 0.14385964 0.01924147]]]]
[[[[0.45692345 0.61498517 0.8125074 ]
   [0.36451542 0.68510985 0.76879174]
   [0.35369605 0.5403194  0.70452   ]
   [0.38356695 0.72149795 0.63645935]
   [0.37684965 0.42428634 0.69243324]
   [0.6859052  0.81489605 0.78171134]
   [0.62323487 0.22881928 0.7439264 ]
   [0.9256844  0.9090549  0.01054902]
   [0.86312294 0.10821906 0.2138522 ]
   [0.879

[[[[0.44524166 0.5953537  0.7522732 ]
   [0.3597924  0.6600418  0.7958266 ]
   [0.35300583 0.5191544  0.666759  ]
   [0.38668925 0.69794077 0.67021286]
   [0.38245723 0.4064008  0.744209  ]
   [0.6765206  0.79522526 0.6694797 ]
   [0.6253316  0.21961257 0.64698195]
   [0.9280925  0.9100417  0.01014009]
   [0.8700882  0.09994923 0.21344903]
   [0.85805625 0.9942304  0.01110601]
   [0.8767929  0.21937947 0.06291884]
   [0.4642646  0.5043573  0.01486339]
   [0.9937164  0.28908238 0.0211879 ]
   [0.65146977 0.9574952  0.00895629]
   [0.81865853 0.3102373  0.01572814]
   [0.5254088  1.0001853  0.01055576]
   [0.86133325 0.25442457 0.01878841]]]]
[[[[0.44524166 0.5953537  0.7522732 ]
   [0.3597924  0.6600418  0.7958266 ]
   [0.35300583 0.5191544  0.666759  ]
   [0.38668925 0.69794077 0.67021286]
   [0.38245723 0.4064008  0.744209  ]
   [0.6765206  0.79522526 0.6694797 ]
   [0.6253316  0.21961257 0.64698195]
   [0.9280925  0.9100417  0.01014009]
   [0.8700882  0.09994923 0.21344903]
   [0.858

[[[[0.45244035 0.5192657  0.57807136]
   [0.36603636 0.57754564 0.7028655 ]
   [0.36654246 0.44204134 0.8051715 ]
   [0.39031488 0.6093619  0.5109987 ]
   [0.39989457 0.33355016 0.6518631 ]
   [0.6232202  0.7325574  0.8144689 ]
   [0.6243387  0.1809064  0.78406084]
   [0.893181   0.8242714  0.22094591]
   [0.8889384  0.04347043 0.11183117]
   [0.88539904 0.8963573  0.07265975]
   [0.86296165 0.2102128  0.02075452]
   [0.4913661  0.49163342 0.01975103]
   [0.9878501  0.24215022 0.01298489]
   [0.87807864 0.79795855 0.01227244]
   [0.62488323 0.17885682 0.01898241]
   [0.735003   1.0031426  0.006555  ]
   [0.72106856 0.02362973 0.02176843]]]]
[[[[0.44261095 0.5098514  0.6548296 ]
   [0.35510787 0.56753176 0.70914805]
   [0.3596283  0.4321084  0.59916985]
   [0.38185188 0.60477936 0.58716744]
   [0.39816827 0.32856023 0.667115  ]
   [0.6296055  0.7281424  0.78973174]
   [0.62526935 0.17574458 0.78557825]
   [0.8845569  0.7951064  0.1244935 ]
   [0.89533687 0.03042629 0.11413489]
   [0.878

[[[[0.43407866 0.48616016 0.68014514]
   [0.35584924 0.53691214 0.5687306 ]
   [0.36186787 0.40708232 0.8656763 ]
   [0.38164353 0.5762065  0.5303164 ]
   [0.40030143 0.3071893  0.78757405]
   [0.63227665 0.7050721  0.77384114]
   [0.6233505  0.1627265  0.79574263]
   [0.88312805 0.80826896 0.11673817]
   [0.89111936 0.02280507 0.15644985]
   [0.86830014 0.88938594 0.06606411]
   [0.87029165 0.19616407 0.01435555]
   [0.37527844 0.67148155 0.01590826]
   [0.24854343 0.2557924  0.02588106]
   [0.8643533  0.87097704 0.00974681]
   [0.6218512  0.15581343 0.02576001]
   [0.7364652  0.886045   0.00215433]
   [0.74616826 0.02006562 0.0157622 ]]]]
[[[[0.44204396 0.4874382  0.67809063]
   [0.3602376  0.5388874  0.67564225]
   [0.36567804 0.4082954  0.8270235 ]
   [0.3820271  0.5766512  0.52711   ]
   [0.39981663 0.30447325 0.7944167 ]
   [0.6164149  0.7027194  0.68813246]
   [0.6269244  0.1604359  0.809413  ]
   [0.8793428  0.83452797 0.2037099 ]
   [0.89680195 0.02267559 0.09371814]
   [0.861